In [ ]:
import torch
from datasets import load_from_disk

In [ ]:
from torch import cuda
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

device = "cuda" if cuda.is_available() else "cpu"

In [ ]:
ds = load_from_disk("bld/python/TrainTest/TrainTest_data/")

## Main Part

Fine-tuning
There are two ways we can implement multi-label classification:

- Creating a custom BERT model that overrides the forward method
- Creating a custom Trainer that overrides the compute_loss method

In [ ]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
# error could be here
cols = ds["train_dataset"].column_names
cols.remove("label")
ds_enc = ds.map(tokenize_and_encode, batched=True, remove_columns=cols)
ds_enc

In [ ]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        inputs.pop("label")  # maybe here error
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(
            logits.view(-1, self.model.config.num_labels),
            labels.float().view(-1, self.model.config.num_labels),
        )
        return (loss, outputs) if return_outputs else loss

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=3).to(
    device,
)

In [ ]:
# not important first
def accuracy_thresh(y_pred, y_true, thresh=0.5, sigmoid=True):
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()
    return ((y_pred > thresh) == y_true.bool()).float().mean().item()

In [ ]:
# not important first
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"accuracy_thresh": accuracy_thresh(predictions, label)}

In [ ]:
batch_size = 8

args = TrainingArguments(
    output_dir="jigsaw",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [ ]:
multi_trainer = MultilabelTrainer(
    model,
    args,
    train_dataset=ds_enc["train_dataset"],
    eval_dataset=ds_enc["val_dataset"],
    compute_metrics=compute_metrics,  # not important for problem
    tokenizer=tokenizer,
)

In [ ]:
multi_trainer.evaluate()

In [ ]:
multi_trainer.train()

#Error MEssage

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[30], line 1
----> 1 multi_trainer.evaluate()

File c:\Users\norma\.conda\envs\EN\Lib\site-packages\transformers\trainer.py:2934, in Trainer.evaluate(self, eval_dataset, ignore_keys, metric_key_prefix)
   2931 start_time = time.time()
   2933 eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
-> 2934 output = eval_loop(
   2935     eval_dataloader,
   2936     description="Evaluation",
   2937     # No point gathering the predictions if there are no metrics, otherwise we defer to
   2938     # self.args.prediction_loss_only
   2939     prediction_loss_only=True if self.compute_metrics is None else None,
   2940     ignore_keys=ignore_keys,
   2941     metric_key_prefix=metric_key_prefix,
   2942 )
   2944 total_batch_size = self.args.eval_batch_size * self.args.world_size
   2945 if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:

File c:\Users\norma\.conda\envs\EN\Lib\site-packages\transformers\trainer.py:3123, in Trainer.evaluation_loop(self, dataloader, description, prediction_loss_only, ignore_keys, metric_key_prefix)
   3120         batch_size = observed_batch_size
   3122 # Prediction step
-> 3123 loss, logits, labels = self.prediction_step(model, inputs, prediction_loss_only, ignore_keys=ignore_keys)
   3124 inputs_decode = self._prepare_input(inputs["input_ids"]) if args.include_inputs_for_metrics else None
...
--> 241     return self.data[item]
    242 elif self._encodings is not None:
    243     return self._encodings[item]

KeyError: 'label'
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...